# EDA of NCAAM / NCAA男子バスケットボールトーナメントの探索的データ解析
This notebook is inspired from the below page.\
https://www.kaggle.com/dhananjay3/ncaam-2021-eda-all-you-need-to-know

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

PATH = '/kaggle/input/ncaam-march-mania-2021/'

for dirname, _, filenames in os.walk('/kaggle/input/ncaam-march-mania-2021/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Data Section 1 - The Basics / 基本データ

This section provides everything you need to build a simple prediction model and submit predictions.
- Team ID's and Team Names
- Tournament seeds since 1984-85 season
- Final scores of all regular season, conference tournament, and NCAA® tournament games since 1984-85 season
- Season-level details including dates and region names
- Example submission file for stage 1

このセクションでは、シンプルな予測モデルを構築して予測を提出するために必要なすべてのデータを提供します。
- チームIDとチーム名
- 1984-85シーズン以降のトーナメントシード
- 1984-85シーズン以降の全てのレギュラーシーズン、カンファレンストーナメント、NCAAトーナメントゲームの最終スコア
- 日付や地域名を含む季節レベルの詳細
- ステージ1の提出ファイルの例

### Data Section 1 file: Teams.csv

This file identifies the different college teams present in the dataset. Each school is uniquely identified by a 4 digit id number. You will not see games present for all teams in all seasons, because the games listing is only for matchups where both teams are Division-I teams. There are 357 teams currently in Division-I, and an overall total of 371 teams in our team listing (each year, some teams might start being Division-I programs, and others might stop being Division-I programs). This year there are four teams that are new to Division I: Bellarmine (TeamID=1469), North Alabama (TeamID=1469), Tarleton State (TeamID=1470), and UC_San Diego (TeamID=1471) and so you will not see any historical data for these teams prior to the current season. In addition, some teams opted not to play during the 2021 season due to the impact of COVID-19 and will not have any games listed.

- **TeamID** - a 4 digit id number, from 1000-1999, uniquely identifying each NCAA® men's team. 
- **TeamName** - a compact spelling of the team's college name, 16 characters or fewer. 
- **FirstD1Season** - the first season in our dataset that the school was a Division-I school. 
- **LastD1Season** - the last season in our dataset that the school was a Division-I school. 

このファイルは、データセットに存在する様々な大学チームを識別します。各大学は、4桁のID番号で一意に識別されます。ゲームリストは両方のチームがディビジョン1チームであるゲームのみを対象としているため、全てのシーズンの全てのチームにゲームが存在するわけではありません。現在ディビジョン1には計357チームが登録されています(毎年、一部のチームはディビジョン1プログラムがスタートするかもしれないし、一部の他のチームはディビジョン1プログラムが終了するかもしれません)。今年はディビジョン1に新しく参加する4つのチームがあります。Ballarmine(チームID=1469)、North Alabama(チームID=1470)、Tarleton State(チームID=1469)、UC_San Diego(チームID=1470)。現在のシーズンより前のこれらのチームの時系列データは存在しません。また、一部のチームはCOVID-19の影響により、2021年のシーズン中にプレイしないことを選択肢、ゲームはリストされません。

- **TeamID** - 1000〜1999の4桁のID番号で、各NCAA男子チームを一意に識別。
- **TeamName** - チームの大学名の省略名。16文字以下。
- **FirstD1Season** - データセット中で大学がディビジョン1であった最初のシーズン。
- **LastD1Season** - データセット中で大学がディビジョン1であった最後のシーズン。

In [ ]:
teams = pd.read_csv(PATH + "MTeams.csv")
teams

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))
sns.countplot(ax=axes[0], data=teams, x='FirstD1Season')
sns.countplot(ax=axes[1], data=teams, x='LastD1Season')
plt.show()

### Data Section 1 file: MSeasons.csv
This file identifies the different seasons included in the historical data, along with certain season-level properties.
- **Season** - indicates the year in which the tournament was played. 
- **DayZero** - tells you the date corresponding to DayNum=0 during that season. All game dates have been aligned upon a common scale so that (each year) the Monday championship game of the men's tournament is on DayNum=154.
- **RegionW, RegionX, Region Y, Region Z** - by our contests' convention, each of the four regions in the final tournament is assigned a letter of W, X, Y, or Z. Whichever region's name comes first alphabetically, that region will be Region W.

このファイルは、特定のシーズンレベルのプロパティとともに、時系列データに含まれる様々なシーズンを識別します。
- **Season** - トーナメントが行われた年を表す。 
- **DayZero** - そのシーズンのDayNum=0に対応する日付を示す。男子トーナメントのMonday championship gameがDayNum=154となるように、全てのゲームの日付は共通のスケールで調整。
- **RegionW, RegionX, Region Y, Region Z** - 最終トーナメントの4つの地域には、それぞれW、X、Y、Zの文字が割り当てられる。アルファベット順で最初に来る地域はRegionWとなる。

In [ ]:
seasons = pd.read_csv(PATH + "MSeasons.csv")
seasons

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 10))
sns.countplot(ax=axes[0][0], data=seasons, x='RegionW')
sns.countplot(ax=axes[0][1], data=seasons, x='RegionX')
sns.countplot(ax=axes[1][0], data=seasons, x='RegionY')
sns.countplot(ax=axes[1][1], data=seasons, x='RegionZ')
plt.show()

### Data Section 1 file: MNCAATourneySeeds.csv

This file identifies the seeds for all teams in each NCAA® tournament, for all seasons of historical data. Thus, there are between 64-68 rows for each year, depending on whether there were any play-in games and how many there were. In recent years the structure has settled at 68 total teams, with four "play-in" games leading to the final field of 64 teams entering Round 1 on Thursday of the first week (by definition, that is DayNum=136 each season). We will not know the seeds of the respective tournament teams, or even exactly which 68 teams it will be, until Selection Sunday on March 14, 2021 (DayNum=132).
- **Season** - the year that the tournament was played in
- **Seed** - this is a 3/4-character identifier of the seed, where the first character is either W, X, Y, or Z (identifying the region the team was in) and the next two digits (either 01, 02, ..., 15, or 16) tell you the seed within the region. For play-in teams, there is a fourth character (a or b) to further distinguish the seeds, since teams that face each other in the play-in games will have seeds with the same first three characters. The "a" and "b" are assigned based on which Team ID is lower numerically.
- **TeamID** - this identifies the id number of the team, as specified in the MTeams.csv file

このファイルは、全てのシーズンのヒストリカルデータについて、各NCAAトーナメントの全てのチームのシードを識別します。したがって、プレーインゲームがあったかどうか、及びその数に応じて、各年で64〜68行になります。最近のデータは計68チームに落ち着き、4つのプレーインゲームが第1週の木曜日の64チームのラウンド1に先立ち行われます(定義上、各シーズンでDayNum=136)。2021年3月14日のセレクションサンデー(DayNum=132)まで、それぞれのトーナメントチームのシード、また正確にどの68チームになるかは分かりません。
- **Season** - トーナメントが行われた年
- **Seed** - シートの3〜4文字の識別子であり、最初の文字はW、X、Y又はZ(チームが所属していた地域を表す)であり、次の2桁(01、02、…、16)はその地域内のシードを表す。プレーインチームの場合、4つ目の文字(a又はb)がある。これは、プレーインゲームで対戦するチームには、最初の3つの文字が同じシードがあるため。aとbは、どちらのチームIDが数値的に低いかに基づいて割り当てられる。 
- **TeamID** - MTeams.csvファイルで指定されているチームIDを表す。

In [ ]:
MNCAATourneySeeds =  pd.read_csv(PATH + "MNCAATourneySeeds.csv")
MNCAATourneySeeds

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 15))
sns.countplot(ax=axes[0], data=MNCAATourneySeeds, y='Season');
sns.countplot(ax=axes[1], data=MNCAATourneySeeds, y='Seed');
plt.show()

### Data Section 1 file: MRegularSeasonCompactResults.csv

This file identifies the game-by-game results for many seasons of historical data, starting with the 1985 season (the first year the NCAA® had a 64-team tournament). For each season, the file includes all games played from DayNum 0 through 132. It is important to realize that the "Regular Season" games are simply defined to be all games played on DayNum=132 or earlier (DayNum=132 is Selection Sunday, and there are always a few conference tournament finals actually played early in the day on Selection Sunday itself). Thus a game played on or before Selection Sunday will show up here whether it was a pre-season tournament, a non-conference game, a regular conference game, a conference tournament game, or whatever.
- **Season** - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs).
- **DayNum** - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "MSeasons.csv" file.
- **WTeamID** - this identifies the id number of the team that won the game, as listed in the "MTeams.csv" file.
- **WScore** - this identifies the number of points scored by the winning team.
- **LTeamID** - this identifies the id number of the team that lost the game.
- **LScore** - this identifies the number of points scored by the losing team.
- **WLoc** - this identifies the "location" of the winning team. If the winning team was the home team, this value will be "H". If the winning team was the visiting team, this value will be "A". If it was played on a neutral court, then this value will be "N".
- **NumOT** - this indicates the number of overtime periods in the game, an integer 0 or higher.

このファイルは、1985年シーズン(NCAAが64チームのトーナメントを開催した最初の年)から始まる、多くのシーズンの時系列データのゲーム毎の結果を表しています。シーズン毎にファイルにはDayNumが0から132までにプレイされて全てのゲームが含まれています。「レギュラーシーズン」ゲームは、DayNum=132以前にプレイされた全てのゲームとして定義されていることを理解することが重要です(DayNum=132はセレクションサンデーであり、実際にその日の早い時間にプレイされるカンファレンストーナメント決勝戦は常にいくつかあります)。
- **Season** - MSeasons.csvにある年(最終トーナメントが行われる年)。
- **DayNum** - 0から132までの整数で、ゲームがプレイされた日。これは、「MSeasns.csv」ファイルの「DayZero」日付からのオフセットを表す。
- **WTeamID** - 「MTeams.csv」ファイルに載っている、ゲームに勝ったチームのID番号。
- **WScore** -ゲームに勝ったチームのスコア。
- **LTeamID** - ゲームに負けたチームのID番号。
- **LScore** - ゲームに負けたチームのスコア。
- **WLoc** - 勝利チームの「場所」の識別子。勝利チームがホームチームの場合は「H」。勝利チームが訪問チームだった場合は「A」。中立的なコートでプレイされた場合は「N」。
- **NumOT** - ０以上の整数で、ゲームの残存期間を表す。

In [ ]:
MRegularSeasonCompactResults = pd.read_csv(PATH + "MRegularSeasonCompactResults.csv")
MRegularSeasonCompactResults

In [ ]:
fig = plt.figure(figsize=(5, 5))
sns.countplot(data=MRegularSeasonCompactResults, x='WLoc')
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 5))
sns.countplot(data=MRegularSeasonCompactResults, x='NumOT')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 5))
sns.countplot(data=MRegularSeasonCompactResults, x='Season')
plt.show()

In [ ]:
fig = plt.figure(figsize=(5, 50))
sns.countplot(data=MRegularSeasonCompactResults, y='WTeamID')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(MRegularSeasonCompactResults.groupby("WTeamID").apply(lambda x: len(x)).values, bins=25)
plt.title("Distribution of total number of wins by each teams")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(MRegularSeasonCompactResults.groupby("LTeamID").apply(lambda x: len(x)).values, bins=25)
plt.title("Distribution of total number of Losses by each teams")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MRegularSeasonCompactResults, x='WScore')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MRegularSeasonCompactResults, x='LScore')
plt.show()

In [ ]:
MRegularSeasonCompactResults["score_difference"] = \
    MRegularSeasonCompactResults["WScore"] - MRegularSeasonCompactResults["LScore"]

fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MRegularSeasonCompactResults, x='score_difference')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MRegularSeasonCompactResults, x='DayNum')
plt.show()

### Data Section 1 file: MNCAATourneyCompactResults.csv

This file identifies the game-by-game NCAA® tournament results for all seasons of historical data. The data is formatted exactly like the MRegularSeasonCompactResults data. All games will show up as neutral site (so WLoc is always N). Note that this tournament game data also includes the play-in games (which always occurred on day 134/135) for those years that had play-in games. Thus each season you will see between 63 and 67 games listed, depending on how many play-in games there were.
\

このファイルは、全てのシーズンの時系列データのゲーム毎のNCAAトーナメントの結果を表します。データは、NRegularSeasonCompactResultsデータと全く同じようにフォーマットされています。全てのゲームは中立的な場所として表されます(WLocは常にN)。このトーナメントゲームのデータには、プレーインゲームがあった年のプレーインゲーム(常に134/135日目に行われる)も含まれていることに注意してください。従って、各シーズンには、プレーインゲームの数に応じて、63〜67のゲームが登録されています。

In [ ]:
MNCAATourneyCompactResults = pd.read_csv(PATH + "MNCAATourneyCompactResults.csv")
MNCAATourneyCompactResults

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.countplot(data=MNCAATourneyCompactResults, x='DayNum')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(MNCAATourneyCompactResults.groupby("WTeamID").apply(lambda x: len(x)).values, bins=25)
plt.title("Distribution of total number of wins by each teams")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(MNCAATourneyCompactResults.groupby("LTeamID").apply(lambda x: len(x)).values, bins=25)
plt.title("Distribution of total number of wins by each teams")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MNCAATourneyCompactResults, x='WScore')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MNCAATourneyCompactResults, x='LScore')
plt.show()

In [ ]:
MNCAATourneyCompactResults["score_difference"] = \
    MNCAATourneyCompactResults["WScore"] - MNCAATourneyCompactResults["LScore"]

fig = plt.figure(figsize=(10, 5))
sns.histplot(data=MNCAATourneyCompactResults, x='score_difference')
plt.show()

### Data Section 1 file: MSampleSubmissionStage1.csv

This file illustrates the submission file format for Stage 1. It is the simplest possible submission: a 50% winning percentage is predicted for each possible matchup.

- **ID** - this is a 14-character string of the format SSSS_XXXX_YYYY, where SSSS is the four digit season number, XXXX is the four-digit TeamID of the lower-ID team, and YYYY is the four-digit TeamID of the higher-ID team.
- **Pred** - this contains the predicted winning percentage for the first team identified in the ID field, the one represented above by XXXX.

このファイルは、ステージ1の提出ファイル形式を表しています。最も単純な提出ファイルで、可能な対戦毎に50%の勝率を予測しています。

- **ID** - SSSS_XXXX_YYYY形式の14文字の文字列です。ここで、SSSSは4桁のシーズン番号、XXXXはTeamIDが低い方の4桁のTeamID、YYYYは高い方の4桁のTeamID。
- **Pred** - ID列で識別された最初のチーム、上記のXXXXで表されたチーム、の予想勝率です。

In [ ]:
MSampleSubmissionStage1 = pd.read_csv(PATH + "MSampleSubmissionStage1.csv")
MSampleSubmissionStage1

## Data Section 2 - Team Box Scores / チームボックススコア

This section provides game-by-game stats at a team level (free throws attempted, defensive rebounds, turnovers, etc.) for all regular season, conference tournament, and NCAA® tournament games since the 2002-03 season.
- **WFGM** - field goals made (by the winning team)
- **WFGA** - field goals attempted (by the winning team)
- **WFGM3** - three pointers made (by the winning team)
- **WFGA3** - three pointers attempted (by the winning team)
- **WFTM** - free throws made (by the winning team)
- **WFTA** - free throws attempted (by the winning team)
- **WOR** - offensive rebounds (pulled by the winning team)
- **WDR** - defensive rebounds (pulled by the winning team)
- **WAst** - assists (by the winning team)
- **WTO** - turnovers committed (by the winning team)
- **WStl** - steals (accomplished by the winning team)
- **WBlk** - blocks (accomplished by the winning team)
- **WPF** - personal fouls committed (by the winning team)

このセクションでは、2002-03シーズン以降の全てのレギュラーシーズン、カンファレンストーナメント、NCAAトーナメントゲームのチームレベルでのゲーム毎の統計(フリースローの回数、ディフェンス時のリバウンド、ターンオーバーなど)を提供します。
- **WFGM** - フィールドゴール(勝利チームによる)
- **WFGA** - フィールドゴールの試み(勝利チームによる)
- **WFGM3** - スリーポイント(勝利チームによる)
- **WFGA3** - スリーポイントの試み(勝利チームによる)
- **WFTM** - フリースロー(勝利チームによる)
- **WFTA** - フリースローの試み(勝利チームによる)
- **WOR** - 攻撃的なリバウンド(勝利チームによる)
- **WDR** - 防御的なリバウンド(勝利チームによる)
- **WAst** - アシスト(勝利チームによる)
- **WTO** - ターンオーバー(勝利チームによる)
- **WStl** - スチール(勝利チームによる)
- **WBlk** - ブロック(勝利チームによる)
- **WPF** - 個人的なファウル(勝利チームによる)

### Data Section 2 file: MRegularSeasonDetailedResults.csv

This file provides team-level box scores for many regular seasons of historical data, starting with the 2003 season. All games listed in the MRegularSeasonCompactResults file since the 2003 season should exactly be present in the MRegularSeasonDetailedResults file.

このファイルは、2003年シーズンから始まる、時系列データの多くの定期的なシーズンのチーム毎のボックススコアを提供します。2003年シーズン以降にMRegularSeasonCompactResultsファイルに載っている全てのゲームは、MRegularSeasonDetailedResultsファイルに含まれています。

In [ ]:
MRegularSeasonDetailedResults = pd.read_csv(PATH + "MRegularSeasonDetailedResults.csv")
MRegularSeasonDetailedResults

In [ ]:
wnew_cols, lnew_cols = ['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR','WAst', 'WTO', 'WStl', 
                        'WBlk', 'WPF'], ['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 
                        'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']

for i, j in zip(wnew_cols, lnew_cols):
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    sns.histplot(ax=axes[0], data=MRegularSeasonDetailedResults, x=i)
    sns.histplot(ax=axes[1], data=MRegularSeasonDetailedResults, x=j)
    plt.show()

### Data Section 2 file: MNCAATourneyDetailedResults.csv

This file provides team-level box scores for many NCAA® tournaments, starting with the 2003 season. All games listed in the MNCAATourneyCompactResults file since the 2003 season should exactly be present in the MNCAATourneyDetailedResults file.

このファイルは、2003年シーズン以降、多くのNCAAトーナメントのチーム毎のボックススコアを提供します。2003年シーズン以降にMNCAATourneyCompactResultsファイルに載っている全てのゲームは、MNCAATourneyDetailedResultsファイルに含まれています。

In [ ]:
MNCAATourneyDetailedResults = pd.read_csv(PATH + "MNCAATourneyDetailedResults.csv")
MNCAATourneyDetailedResults

In [ ]:
for i, j in zip(wnew_cols, lnew_cols):
    fig, axes = plt.subplots(1, 2, figsize=(20, 5))
    sns.histplot(ax=axes[0], data=MNCAATourneyDetailedResults, x=i)
    sns.histplot(ax=axes[1], data=MNCAATourneyDetailedResults, x=j)
    plt.show()

## Data Section 3 - Geography / 地理

This section provides city locations of all regular season, conference tournament, and NCAA® tournament games since the 2009-10 season

このセクションでは、2009-2010年シーズン以降の全てのレギュラーシーズン、カンファレンストーナメント、NCAAトーナメントゲームの年の場所を提供します。

### Data Section 3 file: Cities.csv

This file provides a master list of cities that have been locations for games played. Please notice that the Cities and Conferences files are the only two that don't start with an M; this is because the data files are identical between men's and women's data, so you don't need to maintain separate listings of cities or conferences across the two contests.
- **CityID** - a four-digit ID number uniquely identifying a city.
- **City** - the text name of the city.
- **State** - the state abbreviation of the state that the city is in. In a few rare cases, the game location is not inside one of the 50 U.S. states and so other abbreviations are used.

このファイルは、ゲームがプレイされた場所の都市のマスターリストです。CitiesとConferencesファイルはMで始まらない2つのファイルであることに注意して下さい。これは、データファイルが男性と女性のデータ間で同一であるため、2つのコンテスト間でCitiesまたはConferencesの個別のリストを保持する必要がないためです。
- **CityID** - 都市を一意に識別する4桁のID番号。
- **City** - 都市の名前。
- **State** - 年が存在する州の略語。まれにゲームの場所が米国の50州のいずれにも属さない場合は、他の略語が使用されます。

In [ ]:
Cities = pd.read_csv(PATH + "Cities.csv")
Cities

In [ ]:
fig = plt.figure(figsize=(15, 5))
sns.countplot(data=Cities, x='State')
plt.show()

### Data Section 3 file: MGameCities.csv

This file identifies all games, starting with the 2010 season, along with the city that the game was played in. Games from the regular season, the NCAA® tourney, and other post-season tournaments, are all listed together. There should be no games since the 2010 season where the CityID is not known. Games from the 2009 season and before are not listed in this file.
- **Season, DayNum, WTeamID, LTeamID** - these four columns are sufficient to uniquely identify each game.
- **CRType** - this can be either Regular or NCAA or Secondary. If it is Regular, you can find more about the game in the MRegularSeasonCompactResults.csv and MRegularSeasonDetailedResults.csv files. If it is NCAA, you can find more about the game in the MNCAATourneyCompactResults.csv and MNCAATourneyDetailedResults.csv files. If it is Secondary, you can find more about the game in the MSecondaryTourneyCompactResults file.
- **CityID** - the ID of the city where the game was played, as specified by the CityID column in the Cities.csv file.

このファイルは、2010年シーズンから始まる全てのゲームと、ゲームがプレイされた都市を識別します。レギュラーシーズンのゲーム、NCAAトーナメント、及びその他のシーズン後のトーナメントが全て一緒に纏められています。2010年シーズン以降はCityIDが不明なゲームはありません。2009年シーズン以前のゲームは、このファイルには載っていません。
- **Season, DayNum, WTeamID, LTeamID** - これらの4つの列は、各ゲームを一意に識別することができます。
- **CRType** - これはRegular又はNCAA、Secondaryのいずれかです。Regularの場合は、MRegularSeasonCompactResults.csvファイルとMRegularSeasonDetailedResults.csvファイルでゲームの詳細を確認できます。NCAAの場合、ゲームの詳細はMNCAATourneyCompactResults.csvファイルとMNCAATourneyDetailedResults.csvファイルでゲームの詳細を確認できます。
- **CityID** - Cities.csvファイルのCityID列で指定された、ゲームがプレイされた都市のID。

In [ ]:
MGameCities = pd.read_csv(PATH + "MGameCities.csv")
MGameCities

## Data Section 4 - Public Rankings / パブリックランキング

This section provides weekly team rankings for dozens of top rating systems - Pomeroy, Sagarin, RPI, ESPN, etc., since the 2002-2003 season
\

このセクションでは、2002-2003年のシーズン以降、Pomeroy、Sagarin、RPI、ESPNなどの数十のトップレーティングシステムの毎週のチームレーティングを提供します。

### Data Section 4 file: MMasseyOrdinals.csv

This file lists out rankings (e.g. #1, #2, #3, ..., #N) of teams going back to the 2002-2003 season, under a large number of different ranking system methodologies. 
- **Season** - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs)
- **RankingDayNum** - this integer always ranges from 0 to 133, and is expressed in the same terms as a game's DayNum (where DayZero is found in the MSeasons.csv file). The RankingDayNum is intended to tell you the first day that it is appropriate to use the rankings for predicting games.
- **SystemName** - this is the (usually) 3-letter abbreviation for each distinct ranking system. These systems may evolve from year to year, but as a general rule they retain their meaning across the years.
- **TeamID** - this is the ID of the team being ranked, as described in MTeams.csv.
- **OrdinalRank** - this is the overall ranking of the team in the underlying system. Most systems from recent seasons provide a complete ranking from #1 through #351, but sometimes there are ties and sometimes only a smaller set of rankings is provided, as with the AP's top 25. This year and last year they will typically go up to #353 because two new teams were added to Division I last year.

このファイルには、2002-2003年のシーズンに遡るチームのランキング(例: #1、#2、#3、…、#N)が、多数の異なるランキングシステム手法の下で一覧表示されます。
- **Season** - MSeasons.csvに関連した年です(最終トーナメントが行われる年)。
- **RankingDayNum** - 0から133までの整数で、ゲームのDayNum(DayZeroはMSeasons.csvファイルにあります)と同じ用語で表されます。RankingDayNumは、ゲームの予測にランキングを使用することができる最初の日を示すことを目的としています。
- **SystemName** - (通常)夫々のランキングシステムの3文字の略語です。これらのシステムは年によって変わる可能性がありますが、原則として、それらは何年にも渡ってその意味は変わりません。
- **TeamID** - MTeams.csvで説明されているように、ランク付けされているチームのIDです。
- **OrdinalRank** - システムにおけるチームの全体的なランキングです。最近のシーズンのほとんどのシステムは、#1から#351までの完全なランキングを提供しますが、APのトップ25のように、同数字の場合もあれば、少数のランキングセットしか提供されない場合もあります。

In [ ]:
MMasseyOrdinals = pd.read_csv(PATH + "MMasseyOrdinals.csv")
MMasseyOrdinals

In [ ]:
fig = plt.figure(figsize=(5, 30))
sns.countplot(data=MMasseyOrdinals, y='SystemName')
plt.show()

## Data Section 5 - Supplements / 補足

This section contains additional supporting information, including coaches, conference affiliations, alternative team name spellings, bracket structure, and game results for NIT and other postseason tournaments.
\

このセクションは、コーチ、所属、代替チーム名の綴り、ブラケットの構造、NITやその他のシーズン後のトーナメントの試合結果などの追加の補足情報が含まれています。

### Data Section 5 file: MTeamCoaches.csv

This file indicates the head coach for each team in each season, including a start/finish range of DayNum's to indicate a mid-season coaching change. For scenarios where a team had the same head coach the entire season, they will be listed with a DayNum range of 0 to 154 for that season. For head coaches whose term lasted many seasons, there will be many rows listed, most of which have a DayNum range of 0 to 154 for the corresponding season.
- **Season** - this is the year of the associated entry in MSeasons.csv (the calendar year in which the final tournament occurs)
- **TeamID** - this is the TeamID of the team that was coached, as described in MTeams.csv.
- **FirstDayNum, LastDayNum** - this defines a continuous range of days within the season, during which the indicated coach was the head coach of the team.
- **CoachName** - this is a text representation of the coach's full name, in the format first_last, with underscores substituted in for spaces.

このファイルは、シーズン中のコーチの変更を示すDayNumの開始/終了を含む、各シーズンの各チームのヘッドコーチを示します。チームがシーズン全体で同じヘッドコーチだった場合、そのシーズンのDayNumの範囲は0から154で登録されます。人気が多くのシーズンで続いたヘッドコーチの場合、多くの行が存在し、そのほとんどは対応するシーズンのDayNumの範囲が0から154となります。
- **Season** - MSeasons.csvに関連した年です(最終トーナメントが行われる年)。
- **TeamID** - MTeams.csvで説明されているように、指導されたチームのTeamIDです。
- **FirstDayNum, LastDayNum** - 指定されたコーチがチームのヘッドコーチであったシーズン内の連続した日の範囲です。
- **CoachName** - コーチのフルネームをfirst_lastの形式で表現したもので、スペースの代わりにアンダースコアが使用されています。

In [ ]:
MTeamCoaches = pd.read_csv(PATH + "MTeamCoaches.csv")
MTeamCoaches

### Data Section 5 file: Conferences.csv

This file indicates the Division I conferences that have existed over the years since 1985. Each conference is listed with an abbreviation and a longer name.
- **ConfAbbrev** - this is a short abbreviation for each conference.
- **Description** - this is a longer text name for the conference.

このファイルは、1985年以来何年にもわたって存在していたディヴィジョン1会議を表しています。各カンファレンスは略語と長い名前でリストされています。
- **ConfAbbrev** - 各カンファレンスの短い略語です。
- **Description** - カンファレンスの長い名前です。

In [ ]:
Conferences = pd.read_csv(PATH + "Conferences.csv")
Conferences

### Data Section 5 file: MTeamConferences.csv

This file indicates the conference affiliations for each team during each season.
- **Season** - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs)
- **TeamID** - this identifies the TeamID (as described in MTeams.csv).
- **ConfAbbrev** - this identifies the conference (as described in Conferences.csv).

このファイルは、各シーズン中の各チームの会議の所属を示します。
- **Season** - MSeasons.csvに関連した年です(最終トーナメントが行われる年)。
- **TeamID** - TeamIDです(MTeams.csvで説明されています)。
- **ConfAbbrev** - 会議を識別します(Conferences.csvで説明されています).

In [ ]:
MTeamConferences = pd.read_csv(PATH + "MTeamConferences.csv")
MTeamConferences

### Data Section 5 file: MConferenceTourneyGames.csv

This file indicates which games were part of each year's post-season conference tournaments (all of which finished on Selection Sunday or earlier), starting from the 2001 season. Many of these conference tournament games are held on neutral sites, and many of the games are played by tournament-caliber teams just a few days before the NCAA® tournament.
- **ConfAbbrev** - this identifies the conference (as described in Conferences.csv) that the tournament was for.
- **Season, DayNum, WTeamID, LTeamID** - these four columns are sufficient to uniquely identify each game.

このファイルは、2001年シーズンから毎年のシーズン後のカンファレンストーナメント(全てのセレクションサンデー以前)までのゲームを示しています。これらのカンファレンストーナメントゲームの多くは中立的な場所で開催され、ゲームの多くはNCAAトーナメントのほんの数日前にトーナメントキャリバーチームによってプレイされます。
- **ConfAbbrev** - トーナメントの対象となったカンファレンス(Conferences.csvで説明されている)を識別します。
- **Season, DayNum, WTeamID, LTeamID** - これら4つの列は、各ゲームを一意に識別することができます。

In [ ]:
MConferenceTourneyGames = pd.read_csv(PATH + "MConferenceTourneyGames.csv")
MConferenceTourneyGames

### Data Section 5 file: MSecondaryTourneyTeams.csv

This file identifies the teams that participated in post-season tournaments other than the NCAA® Tournament (such events would run in parallel with the NCAA® Tournament). These are teams that were not invited to the NCAA® Tournament and instead were invited to some other tournament, of which the NIT is the most prominent tournament, but there have also been the CBI, CIT, and Vegas 16 (V16) at various points in recent years.
- **Season** - this is the year of the associated entry in MSeasons.csv (the year in which the post-season tournament was played)
- **SecondaryTourney** - this is the abbreviation of the tournament, either NIT, CBI, CIT, or V16 (which stands for Vegas 16).
- **TeamID** - this identifies the TeamID that participated in the tournament (as described in MTeams.csv).

このファイルは、NCAAトーナメント以外のポストシーズントーナメントに参加したチームを識別します(このようなイベントはNCAAトーナメントと並行して行われます)。これらはNCAAトーナメントに招待されなかったチームであり、代わりにNITが最も有名なトーナメントである他のトーナメントに招待されましたが、近年ではCBIやCIT,Vegas 16(V16)もあります。
- **Season** -  MSeasons.csvに関連した年です(最終トーナメントが行われる年)。
- **SecondaryTourney** - これはトーナメントの略語で、NIT、CBI、CIT、又はV16(Vegas 16の略)のいずれかです。
- **TeamID** - トーナメントに参加したチームのIDです(MTeams.csvで説明されています)。

In [ ]:
MSecondaryTourneyTeams = pd.read_csv(PATH + "MSecondaryTourneyTeams.csv")
MSecondaryTourneyTeams

### Data Section 5 file: MSecondaryTourneyCompactResults.csv

This file indicates the final scores for the tournament games of "secondary" post-season tournaments: the NIT, CBI, CIT, and Vegas 16. The detailed results (team box scores) have not been assembled for these games.
- **SecondaryTourney** - this is the abbreviation of the tournament, either NIT, CBI, CIT, or V16 (which stands for Vegas 16).

このファイルは、「セカンダリ」ポストシーズントーナメント(NIT、CBI、CIT、及びVegas 16)の最終スコアを示しています。これらのゲームの詳細な結果(チームボックススコア)は収集されていません。
- **SecondaryTourney** - これはトーナメントの略語で、NIT、CBI、CIT、又はV16(Vegas 16の略)のいずれかです。

In [ ]:
MSecondaryTourneyCompactResults = pd.read_csv(PATH + "MSecondaryTourneyCompactResults.csv")
MSecondaryTourneyCompactResults

### Data Section 5 file: MTeamSpellings.csv

This file indicates alternative spellings of many team names. It is intended for use in associating external spellings against our own TeamID numbers, thereby helping to relate the external data properly with our datasets.
- **TeamNameSpelling** - this is the spelling of the team name. 
- **TeamID** - this identifies the TeamID for the team that has the alternative spelling (as described in MTeams.csv).

このファイルは、多くのチーム名の代替スペルを示しています。これは、外部のスペルを我々のTeamIDに関連づけて、外部データをデータセットに適切に関連づけるのに役立つことを目的としています。
- **TeamNameSpelling** - チーム名のスペルです。
- **TeamID** - 代替スペルを持つチームのTeamIDです(MTeams.csvで説明されています)。

In [ ]:
MTeamSpellings = pd.read_csv(PATH + "MTeamSpellings.csv", encoding = "ISO-8859-1")
MTeamSpellings

### Data Section 5 file: MNCAATourneySlots

This file identifies the mechanism by which teams are paired against each other, depending upon their seeds, as the tournament proceeds through its rounds. 
- **Season** - this is the year of the associated entry in MSeasons.csv (the year in which the final tournament occurs)
- **Slot** - this uniquely identifies one of the tournament games. For play-in games, it is a three-character string identifying the seed fulfilled by the winning team, such as W16 or Z13. For regular tournament games, it is a four-character string, where the first two characters tell you which round the game is (R1, R2, R3, R4, R5, or R6) and the second two characters tell you the expected seed of the favored team. 
- **StrongSeed** - this indicates the expected stronger-seeded team that plays in this game. 
- **WeakSeed** - this indicates the expected weaker-seeded team that plays in this game, assuming all favored teams have won so far. 

このファイルは、トーナメントがラウンドを進むときに、シードに応じてチームが互いにペアになるメカニズムを識別します。
- **Season** - MSeasons.csvに関連した年です(最終トーナメントが行われる年)。
- **Slot** - トーナメントゲームを一意に識別します。プレーインゲームの場合、これはW16やZ13など、優勝チームが達成したシードを識別する3文字の文字列です。通常のトーナメントゲームの場合、これは4文字の文字列で、最初の2文字はゲームのラウンド(R1、R2、R3、R4、R5、R6)を示し、次の2文字は予想されるシードを示します。 
- **StrongSeed** - ゲームでプレーするより強いシードのチームを示します。
- **WeakSeed** - 全ての優先チームがこれまでに勝ったと仮定して、弱いシードのチームを示します。

In [ ]:
MNCAATourneySlots = pd.read_csv(PATH + "MNCAATourneySlots.csv")
MNCAATourneySlots

### Data Section 5 file: MNCAATourneySeedRoundSlots.csv

This file helps to represent the bracket structure in any given year. No matter where the play-in seeds are located, we can always know, for a given tournament seed, exactly what bracket slot they would be playing in, on each possible game round, and what the possible DayNum values would be for that round. 
- **Seed** - this is the tournament seed of the team.
- **GameRound** - this is the round during the tournament that the game would occur in, where Round 0 (zero) is for the play-in games, Rounds 1/2 are for the first weekend, Rounds 3/4 are for the second weekend, and Rounds 5/6 are the national semifinals and finals.
- **GameSlot** - this is the game slot that the team would be playing in, during the given GameRound. The naming convention for slots is described above, in the definition of the MNCAATourneySlots file.
- **EarlyDayNum, LateDayNum**- these fields describe the earliest possible, and latest possible, DayNums that the game might be played on.

このファイルは、特定の年のブラケット構造を表すのに役立ちます。プレイインシードがどこにあるかに関係なく、特定のトーナメントシードについて、可能な各ゲームラウンドで、どのブラケットスロットでプレイするか、及びそのラウンドで可能なDayNumの値を常に知ることができます。
- **Seed** - チームのトーナメントシードです。
- **GameRound** - ゲームが行われるトーナメント中のラウンドです。ラウンド0はプレイインゲーム用、ラウンド1/2は最初の週末用、ラウンド3/4は2番目の週末用です。ラウンド5/6は全国準決勝と決勝です。
- **GameSlot** - 特定のGameRound中にチームがプレイするゲームスロットです。スロットの命名規則は、MNCAATourneySlotsファイルの定義で前述されています。
- **EarlyDayNum, LateDayNum**- これらのフィールドは、ゲームがプレイされる可能性のある、可能な限り早い、及び可能な限り遅いDayNumを示します。

In [ ]:
MNCAATourneySeedRoundSlots = pd.read_csv(PATH + "MNCAATourneySeedRoundSlots.csv")
MNCAATourneySeedRoundSlots